# Sagemaker Walkthrough

In this notebook, we will go through the steps of training and deploying a model using AWS Sagemaker.

We will be using the [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) module throughout this notebook. 

We will use two different types of models for our predictions, a model built using scikit-learn, and one using an implementation of XGBoost available from sagemaker, including performing hyperparameter tuning.

## Step 1: Initial Setup

First, let's bring our data over. It is the familiar King County Housing dataset, and is currently sitting in an s3 bucket with path *s3://nss-ds3/datasets/kc_house_data.csv*

Let's use the *boto3* library to fetch our data. We'll do some tricks and read it directly to a pandas dataframe.

In [44]:
import pandas as pd
import boto3
import io

In [100]:
s3 = boto3.client('s3')
obj1 = s3.get_object(Bucket='nss-ds3', Key='datasets/kaggle/train.csv')
obj2 = s3.get_object(Bucket='nss-ds3', Key='datasets/kaggle/test.csv')
train = pd.read_csv(io.BytesIO(obj1['Body'].read()))
kaggle_test = pd.read_csv(io.BytesIO(obj2['Body'].read()))


In [67]:
#train = train[['time', 'signal']]

Now that we have our data, we need to set create a sagemaker session and get our execution role. We will need to pass these as arguments when we create our models later.

In [101]:
# S3 prefix
prefix = 'sagemaker_kaggle_dalila'

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

## Preparing Data for Modeling

We'll do some minor preprocessing of our data and then export the results to a csv. Notice that we will put the column we want to predict at the front, because this is what the XGBoost model will expect.

In [102]:
train = train[['open_channels','time','signal']]

In [103]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(train, test_size = 0.2)

#X = train[['time', 'signal']]
#y = train[['open_channels']]
#X_train, y_train, X_test, y_test = train_test_split(X, y)


In [117]:
import os

os.makedirs('./data', exist_ok=True)
train.to_csv('data/kaggs_train.csv', index = False)

#X.to_csv('data/kaggs_train_X.csv', index = False)

Now, we are going to put our training data into the s3 bucket for our sagemaker instance. The following cell will take the contents of the data folder (currently our train.csv) and copy them over to a folder sm-housing/data in s3.

In [122]:
prefix = 'sm-kaggle'

WORK_DIRECTORY = 'data'

train_input = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY) )

Now, we are going to fit our scikit-learn model. For this to work, we need to upload the training script to our notebook instance, as the SKLearn class will be looking for it to use as an entry point.

In [123]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'housing_script_rf.py'

sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session)

Now, we can fit our model, telling it where to look for the training data.

In [124]:
sklearn.fit({'train': train_input})
#sklearn.fit({'kaggs_train_X': train_input})

2020-03-22 16:48:18 Starting - Starting the training job...
2020-03-22 16:48:21 Starting - Launching requested ML instances...
2020-03-22 16:49:15 Starting - Preparing the instances for training......
2020-03-22 16:49:58 Downloading - Downloading input data...
2020-03-22 16:50:41 Training - Training image download completed. Training in progress..2020-03-22 16:50:42,643 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-03-22 16:50:42,645 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-22 16:50:42,655 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-03-22 16:50:42,932 sagemaker-containers INFO     Module housing_script_rf does not provide a setup.py. 
Generating setup.py
2020-03-22 16:50:42,932 sagemaker-containers INFO     Generating setup.cfg
2020-03-22 16:50:42,932 sagemaker-containers INFO     Generating MANIFEST.in
2020-03-22 16:50:42,933 sagemaker-containers INFO     Inst

Now that our model is trained, we have two options. First, we can retrieve the actual model (which can also be downloaded to your local machine). Second, we can deploy the model. For our scikit-learn model, will go with the first option. We'll look at the second one for the xgboost model that we'll do next.

Let's see where the model output is stored:

In [125]:
boto3.client('sagemaker').describe_training_job(
    TrainingJobName=sklearn.latest_training_job.job_name)['ModelArtifacts']['S3ModelArtifacts']

's3://sagemaker-us-east-2-339692866702/sagemaker-scikit-learn-2020-03-22-16-48-18-491/output/model.tar.gz'

In [126]:
## Sub in what you see above and then uncomment the following lines
s3.download_file(Bucket = 'sagemaker-us-east-2-339692866702', 
                 Key = 'sagemaker-scikit-learn-2020-03-22-16-48-18-491/output/model.tar.gz',
                 Filename = 'model.tar.gz')

In [127]:
import tarfile

tar = tarfile.open('model.tar.gz')
tar.extractall()
tar.close()

In [128]:
from sklearn.externals import joblib

In [129]:
rf_model = joblib.load('model.joblib')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.20.0 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.20.0 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Let's see how it does on the test data.

In [130]:
from sklearn.metrics import mean_absolute_error

In [131]:
y_pred = rf_model.predict(test.iloc[:,1:])

Recall that the mean absolute error tells us how far off our predictions are, on average, from the true values.

In [132]:
mean_absolute_error(test.iloc[:,0], y_pred)

0.5994199083732523

In [137]:
kaggle_test.head()

,time,signal
0,500.0001,-2.6498
1,500.0002,-2.8494
2,500.0003,-2.8600
3,500.0004,-2.4350
4,500.0005,-2.6155


In [133]:
#kaggle_test = kaggle_test.values.reshape(-1,1)
kaggle_pred = rf_model.predict(kaggle_test)

In [140]:
kaggle_output = kaggle_pred.astype('int')

In [143]:
obj3 = s3.get_object(Bucket='nss-ds3', Key='datasets/kaggle/sample_submission.csv')
sample_sub = pd.read_csv(io.BytesIO(obj3['Body'].read()))

In [145]:
sample_sub.time = sample_sub.time.apply(lambda x: '{:.4f}'.format(x))

In [146]:
sample_sub.open_channels = kaggle_output

In [147]:
sample_sub.to_csv('kaggle_submission.csv', index=False)

## Section 2: Using sagemaker's XGBoost Algorithm

Now, we will see how to use the XGBoost algorithm.

We need to get the location of the xgboost model. This is done with the get_image_uri method.

In [23]:
region = boto3.Session().region_name    
smclient = boto3.Session().client('sagemaker')

bucket = sagemaker.Session().default_bucket()   
prefix = 'kaggle_xgb'

In [24]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(region, 'xgboost', repo_version='0.90-1')

First, create an Estimator instance, pointing to the xgboost container.

In [25]:
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.m4.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)

Set the hyperparameters for our model. A list of available hyperparameters and what they represent is available here:https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html

In [26]:
xgb.set_hyperparameters(
    num_round=100,
    rate_drop=0.3,
    alpha = 0.25,
    lambda = 1)

Move our training data to s3.

In [27]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('data/train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

In [28]:
xgb.fit({'train': s3_input_train})

2020-03-14 00:43:29 Starting - Starting the training job...
2020-03-14 00:43:32 Starting - Launching requested ML instances......
2020-03-14 00:44:36 Starting - Preparing the instances for training......
2020-03-14 00:45:38 Downloading - Downloading input data...
2020-03-14 00:46:30 Training - Training image download completed. Training in progress...INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[00:46:33] 17291x85 matrix with 1469735 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single node training.
INFO:root:Train matrix has 17291 rows
[0]#011train-rmse:471337
[1]#011train-rmse:349028
[2]#011train-rmse:265043
[3]#011train-rmse:207124
[4]#011trai

Now let's deploy our model. This will create an endpoint holding our model.

In [29]:
predictor = xgb.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

---------------!

Now that we have our model deployed, we can use it to make predictions.

Notice that it does take a little bit of work to ensure that we are sending our data to the model in the correct format.

In [30]:
import numpy as np
from sagemaker.predictor import csv_serializer

In [ ]:
predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [31]:
y_pred = predict(test.values[:, 1:])

In [32]:
mean_absolute_error(test.iloc[:,0], y_pred)

71036.91989792968

Warning: if you are making a large number of predictions, you will have to do it in batches, because is a cap on how much data you can pass to an endpoint at a time.